# Project 4: Wine Classification

In [ ]:
# Add all the necessary imports here

## Introduction

As you might know, some fancy people have standards when it comes to wine. They insist that not all fermented grape juice is equal. In this project, we will try to predict whether a wine will be "good" or not based on a number of measurements.

Let us start by downloading the relevant datasets: one for red wine and another for white.

In [ ]:
# The two URL's for the datasets:
url_red_wine = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
url_white_wine = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'

# Load the dataset by reading the CSV into a data structure. Notice the "sep" parameter in the function.
# Why is it there?

red_wine_data = pd.read_csv(url_red_wine, sep = ';');
white_wine_data = pd.read_csv(url_white_wine, sep = ';');

### Question 1

Now that the data is loaded, show the first 10 entries in each of the two datasets. Remeber, when we use *pandas* to read the data, the output is a dataframe which has all kinds of convenient functions.

Each of the data sets has samples of wine with corresponding measurements and a "quality" label ranging from 1 to 10.

### Question 2:

For each of the datasets, show the count for each quality score. Think of a good way of presenting it so the reader can quickly see how the scores are distributed. *Hint: You need to look only at the quality column for each dataset and find out how many times each number shows up.*

We would like to be able to classify our wines as "good" (quality 7 or above) or "not" (otherwise).

## Data Preprocessing

### Question 3

Add a column to both datasets that labels the wine as good or not. Remember, later on we will want to use this label in our neural networks so think of how to make it computationally friendly.
*Hint: Take a look here https://stackoverflow.com/questions/26886653/pandas-create-new-column-based-on-values-from-other-columns*

### Question 4

The next step is to create the training and the test sets. Here, we will use 75% of the datasets for training and the remaining portion for testing.

#### (a)

We don't know whether the items in the table are arranged in any way. Shuffle them. *Hint: search 'pandas shuffle'. Look at the stackoverflow result.*

#### (b)
Take 75% of the datasets to create the training sets. Use the remaining data for the test sets. *Hint: look at how we did this for the Iris dataset*

### Question 5

For all the dataseta, create the "measurements" sets and the corresponding labels. You should have 8 items in total: measurements + labels for training and testing for two wine types.

## Model Training

### Question 6

Create and compile two models using keras: one for red wine, another for white. The architecture is up to you. What is the smallest network you need to get decent results?

### Question 7:

Train the models. How well to they perform on the training and test data?

## Question 8:

How do you know you are not overfitting/underfitting
Explain why over/underfitting would be a problem

## Question 9:

Accuracy does not give the full picture. Analyze your predictions based on other evaluation metrics: **precision** and **recall**. Compare and contrast the two metrics. What do their scores tell you about your model results and how it can be improved?

## Question 10:

How do you think you precision and recall results would change based on the proportional breakdown between "good" and "bad" wines in your datasets? How would you solve the problem of an imbalanced dataset? *Hint: imagine your dataset only has good/bad wines and your model becomes great at predicting only one category. Google "imbalanced datasets".* 

**You do not need to implement the different scenarios or the fix. Simply provide a discussion.**